In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")


In [0]:
%%sh
ls /tmp/

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=apple-store/

In [0]:

df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/")
df1.show(1, True ,True)
df1.createOrReplaceTempView('app_detail')

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5432 -U app_tomcat -d aa << EOF
\d aa_app;
-- SELECT id, name, company FROM aa_app LIMIT 1;
-- SELECT id, name, company_id FROM aa_app WHERE id=383004611 LIMIT 1;
EOF

In [0]:
%%sh
postgres --version

In [0]:
%%sh
# host : pg-n-slave-4edc958bda550bbc.elb.us-east-1.amazonaws.com
# User:app_bdp_usage_qa
# Port:5433
# Database names:
# aa_android
PGPASSWORD='2mHdFW6%#REu' psql -h pg-n-slave-4edc958bda550bbc.elb.us-east-1.amazonaws.com -p 5433 -U app_bdp_usage_qa -d aa_android << EOF
\d app;
EOF

In [0]:
%%sh
# PG_AA_HOSTS = [('10.2.26.136', 6432)]
# PG_AA_NAME = 'aa'
# PG_AA_ACCESS_ID = 'app_bdp_usage_qa'
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136 -p 6432 -U app_bdp_usage_qa -d aa << EOF
\d aa_app;
EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5432 -U app_tomcat -d aa << EOF
-- \d aa_app;
-- SELECT id, company_id FROM aa_app LIMIT 1;
SELECT count(id) FROM aa_app LIMIT 1;
-- SELECT * FROM aa_app WHERE id=383004611 LIMIT 1;
-- select count(1) from aa_app where company_id is NULL and status=1 LIMIT 10;
-- SELECT count(*) FROM aa_app WHERE status=1 LIMIT 1;
-- SELECT count(1) FROM aa_app LIMIT 1;
-- SELECT COUNT(1) FROM aa_app WHERE last_updated <= '2020-05-18 15:54:59.060316' LIMIT 1;
-- SELECT COUNT(1) FROM aa_app where created <= '2020-05-18 15:54:59.060316';
-- SELECT id, publisher FROM aa_app where publisher is null LIMIT 10;
-- SELECT id, company, company_id FROM aa_app LIMIT 1;
-- select id, company, company_id from aa_app limit 1
-- offset floor(random() * 50000);
--SELECT *
--FROM aa_app TABLESAMPLE SYSTEM (50) LIMIT 1;
SELECT reltuples
FROM pg_class
WHERE relname='aa_app';
EOF

In [0]:
%%sh
PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5433 -U app_tomcat -d aa_android << EOF
\d app;
-- SELECT id, name, company FROM app LIMIT 1;
-- SELECT id, name, company FROM app WHERE id=383004611 LIMIT 1;
SELECT count(*) FROM app LIMIT 1;
-- SELECT count(*) FROM app WHERE status=1 LIMIT 1;
-- SELECT max(last_updated) FROM app WHERE status=1 LIMIT 1;
-- SELECT COUNT(1) FROM app WHERE last_updated::text LIKE '2020-05-18%' LIMIT 1;
SELECT reltuples
FROM pg_class
WHERE relname='app';
EOF

In [0]:

'''
    id     | company_id 
-----------+------------
 876308786 |  539408775
'''
df2 = spark.sql(
    "SELECT * "
    "FROM app_detail "
    "WHERE id='876308786'"
    )
df2.show(10, False, True)

In [0]:
%%sh


In [0]:

spark.sql(
    "SELECT count(*) "
    "FROM app_detail "
    "WHERE market_code='apple-store' AND last_updated <= '2020-05-18 15:54:59.060316'"
    ).show()

In [0]:

spark.sql(
    "SELECT count(*) "
    "FROM app_detail "
    "WHERE market_code='google-play' AND last_updated LIKE '2020-05-18%'"
    ).show()

In [0]:

spark.sql(
    "SELECT count(*) "
    "FROM app_detail "
    "WHERE status=1 and market_code='apple-store'"
    ).show()

In [0]:

spark.sql(
    "SELECT count(*) "
    "FROM app_detail "
    "WHERE status=1 and market_code='google-play'"
    ).show()

In [0]:

spark.sql(
    "SELECT max(last_updated) "
    "FROM app_detail "
    "WHERE status=1"
    ).show(10, False)

In [0]:
 
spark.sql(
    "SELECT count(*) "
    "FROM app_detail "
    "WHERE market_code='apple-store' AND created <= '2020-05-18 15:54:59.060316'"
    ).show(10, False)

In [0]:

from datetime import date, datetime

from applications.data_validation_v1.constants import APP_DETAIL_UNIFIED_URN, GOOGLE_PLAY, \
    APPLE_STORE, IOS_FIELDS_MAPPING, GOOGLE_PLAY_FIELDS_MAPPING, urn
from applications.data_validation_v1.utils import get_unified_path_app_details, pg_settings, \
    get_country_id_from_country_code, get_ios_unified_category_id, get_gp_unified_category_id_from_id
from aadatapipelinecore.core.fs import Conf
from aadatapipelinecore.core.fs.device import S3Bucket
from aadatapipelinecore.core.utils.spark import canned_spark, stop

def test_ios_count_of_raw_data_and_unified_data(spark):
    s3 = S3Bucket(Conf(bucket_name='b2b-prod-int-data-pipeline-unified'))
    max_ios_created_list = []
    ios_batch_list = s3.list('unified/app-tech.store.app.v1/dimension/market_code={}/'.format(APPLE_STORE))
    for item in ios_batch_list:
        created = spark.read.parquet('s3://b2b-prod-int-data-pipeline-unified/' + item).select('created').rdd.max()[0]
        max_ios_created_list.append(created)
    unified_ios_create = min(max_ios_created_list)
    ios_max_created = spark.read.parquet(get_unified_path_app_details(APP_DETAIL_UNIFIED_URN, APPLE_STORE)).filter(
        "created <= '{}'".format(unified_ios_create)).count()
    ios_created_sql = "SELECT COUNT(1) FROM aa_app where created <= '{}'".format(unified_ios_create)
    ios_extra_count = pg_settings(urn, 'aa', ios_created_sql)
    assert ios_max_created == ios_extra_count[0][0], \
        'ios_max_created:{}, ios_extra_count[0][0]: {}'.format(ios_max_created, ios_extra_count[0][0])


In [0]:

from applications.db_check_v1.common.db_check_utils import query_df, query, pg_settings, etl_skip
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5432 -U app_tomcat -d aa << EOF
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5433 -U app_tomcat -d aa_android << EOF
def pg_settings():
    return (
        "dbname='{db}' user='{user}' password='{password}' "
        "host='{host}' port='{port}'".format(
            db= 'aa_android',
            user= 'app_tomcat',
            host= '54.210.244.2',
            password= 'TMV!PYT02X*w',
            port= '5433'
        )
    )
_pg_dsn = pg_settings()
sql1 = "SELECT id, name, company_id FROM app WHERE id=383004611 LIMIT 1"
ios_res = query(_pg_dsn, sql1)
print ios_res

In [0]:

from applications.db_check_v1.common.db_check_utils import query_df, query, pg_settings, etl_skip
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5432 -U app_tomcat -d aa << EOF
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5433 -U app_tomcat -d aa_android << EOF
# IOS
_pg_dsn = pg_settings('aa')
sql_ios = "SELECT id, name, company_id FROM aa_app WHERE id=383004611 LIMIT 1"
ios_res = query(_pg_dsn, sql_ios)
print ios_res

In [0]:

from applications.db_check_v1.common.db_check_utils import query_df, query, pg_settings, etl_skip
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5432 -U app_tomcat -d aa << EOF
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5433 -U app_tomcat -d aa_android << EOF
# Android
_pg_dsn = pg_settings('aa_android')
sql_gp = "SELECT id, name, company_id FROM app WHERE id=383004611 LIMIT 1"
gp_res = query(_pg_dsn, sql_gp)
print gp_res

In [0]:

max_ios_created_list = []
created = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=apple-store/").select('created').rdd.max()[0]
max_ios_created_list.append(created)
unified_ios_create = min(max_ios_created_list)

In [0]:

print unified_ios_create

In [0]:

from aadatapipelinecore.core.fs import Conf
from aadatapipelinecore.core.fs.device import S3Bucket
from aadatapipelinecore.core.utils.spark import canned_spark, stop

In [0]:

from aadatapipelinecore.core.fs import Conf
from aadatapipelinecore.core.fs.device import S3Bucket, specified_bucket
from aadatapipelinecore.core.fs.device import unified_bucket
from aadatapipelinecore.core.fs.driver.unified import read
from aadatapipelinecore.core.fs.router import UnifiedRouter
from aadatapipelinecore.core.pipeline import type_
from aadatapipelinecore.core.utils import category
from aadatapipelinecore.core.loader import es
import psycopg2
from aadatapipelinecore.core.urn import Urn
from conf import settings

def get_unified_path_app_details(_urn, market_code):
    router = UnifiedRouter(_urn)
    logic_key = router.logic_key()
    _path = unified_bucket().url(logic_key, False)
    return "{}/market_code={}/".format(_path, market_code)
    
APP_DETAIL_UNIFIED_URN = Urn(
    namespace='store.app.v1',
    data_type='dimension'
)
urn = Urn(
    namespace='app-qa.data.validation.v1',
    owner='app-qa'
)

def pg_settings(_urn, schema, sql):
    template = "PG_" + schema.upper() + "_{property}"
    host, port = getattr(settings, template.format(property='HOSTS'))[0]
    conn = psycopg2.connect(
        database=getattr(settings, template.format(property='NAME')),
        user=getattr(settings, template.format(property='ACCESS_ID')),
        password=getattr(settings, template.format(property='SECRET_KEY')),
        host=host,
        port=port)
    cur = conn.cursor()
    cur.execute(sql)
    res = cur.fetchall()
    cur.close()
    conn.close()
    return res

In [0]:

from aadatapipelinecore.core.fs import Conf
from aadatapipelinecore.core.fs.device import S3Bucket

In [0]:

s3 = S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-store-free'))
max_ios_created_list = []
ios_batch_list = s3.list('unified/store.app.v1/dimension/market_code={}/'.format('apple-store'))
for item in ios_batch_list:
    created = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/' + item).select('created').rdd.max()[0]
    max_ios_created_list.append(created)
print max_ios_created_list


In [0]:

unified_ios_create = max(max_ios_created_list)
print unified_ios_create
ios_max_created = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=apple-store/').filter(
    "created <= '{}'".format(unified_ios_create)).count()
print ios_max_created


In [0]:

print unified_ios_create
ios_created_sql = "SELECT COUNT(1) FROM aa_app where created <= '{}'".format(unified_ios_create)
ios_extra_count = pg_settings(urn, 'aa', ios_created_sql)
print ios_extra_count[0][0]
# assert ios_max_created == ios_extra_count[0][0], \
    # 'ios_max_created:{}, ios_extra_count[0][0]: {}'.format(ios_max_created, ios_extra_count[0][0])

In [0]:

print max_ios_created_list[0]

In [0]:

ios_max_date = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=apple-store/').select('created').rdd.max()[0]
print ios_max_date
ios_max_created = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=apple-store/').filter(
    "created <= '{}'".format(ios_max_date)).count()
print ios_max_created

In [0]:

gp_max_date = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=google-play/').select('created').rdd.max()[0]
print gp_max_date
gp_max_created = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/market_code=google-play/').filter(
    "created <= '{}'".format(gp_max_date)).count()
print gp_max_created

In [0]:

from applications.db_check_v1.common.db_check_utils import query_df, query, pg_settings, etl_skip
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5432 -U app_tomcat -d aa << EOF
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5433 -U app_tomcat -d aa_android << EOF
# IOS
_pg_dsn = pg_settings('aa')
sql_ios = "SELECT COUNT(id) FROM aa_app where created <= '{}'".format(ios_max_date)
ios_res = query(_pg_dsn, sql_ios)
print ios_res[0]

In [0]:

from applications.db_check_v1.common.db_check_utils import query_df, query, pg_settings, etl_skip
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5432 -U app_tomcat -d aa << EOF
# PGPASSWORD='TMV!PYT02X*w' psql -h 54.210.244.2 -p 5433 -U app_tomcat -d aa_android << EOF
# Android
_pg_dsn = pg_settings('aa_android')
sql_gp = "SELECT COUNT(id) FROM app where created <= '{}'".format(gp_max_date)
gp_res = query(_pg_dsn, sql_gp)
print gp_res[0]

In [0]:

IOS_FIELDS_MAPPING = {0: 'id',
                      1: 'name',
                      2: 'publisher',
                      3: 'release_date',
                      4: 'artwork_url',
                      5: 'slug',
                      6: 'scrape_reviews',
                      7: 'price',
                      8: 'version',
                      9: 'description',
                      10: 'date_scraped',
                      11: 'scrape_failed',
                      12: 'country_code',
                      13: 'dead',
                      14: 'publisher_id',
                      15: 'category_id',
                      16: 'ss_urls',
                      17: 'sku',
                      18: 'size',
                      19: 'req_version',
                      20: 'req_device',
                      21: 'web_urls',
                      22: 'has_game_center',
                      23: 'is_mac',
                      24: 'languages',
                      25: 'created',
                      26: 'support_url',
                      27: 'license_url',
                      28: 'content_rating',
                      29: 'link_apps',
                      30: 'scrape_review_delay',
                      31: 'requirements',
                      32: 'privacy_policy_url',
                      33: 'app_store_notes',
                      34: 'last_updated',
                      35: 'has_iap',
                      36: 'status',
                      37: 'bundle_id',
                      38: 'product_type',
                      39: 'bundle_product_count',
                      40: 'family_sharing',
                      41: 'purchased_separately_price',
                      42: 'seller',
                      43: 'required_devices',
                      44: 'current_release_date',
                      45: 'original_price',
                      46: 'sensitive_status',
                      47: 'has_imsg',
                      48: 'is_hidden_from_springboard',
                      49: 'subtitle',
                      50: 'promotional_text',
                      51: 'editorial_badge_type',
                      52: 'editorial_badge_name',
                      53: 'only_32_bit'}

In [0]:

import unittest
from applications.db_check_v1.common.db_check_utils import query, pg_settings
from applications.db_check_v1.common.base_test import PipelineTest
from aadatapipelinecore.core.utils.spark import create_spark


class TestGameIQMarketSizeFullRestatement(PipelineTest):
    trigger_date_config = ('* * * * *', 0)
    base_path = "s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/"
    @classmethod
    def setUpClass(cls):
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext

    # def test_ios_count_of_raw_data_and_unified_data(self):
    #     ios_df = self.spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/'
    #                                      'store.app.v1/dimension/market_code=apple-store/')
    #     ios_max_date = ios_df.select('created').rdd.max()[0]
    #     print ios_max_date
    #     ios_max_created = ios_df.filter(
    #         "created <= '{}'".format(ios_max_date)).count()
    #     print ios_max_created

    #     gp_df = self.spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/'
    #                                     'store.app.v1/dimension/market_code=google-play/')
    #     gp_max_date = gp_df.select('created').rdd.max()[0]
    #     print gp_max_date
    #     gp_max_created = gp_df.filter(
    #         "created <= '{}'".format(gp_max_date)).count()
    #     print gp_max_created

    #     # IOS
    #     _pg_dsn = pg_settings('aa')
    #     sql_ios = "SELECT COUNT(1) FROM aa_app where created <= '{}'".format(ios_max_date)
    #     ios_res = query(_pg_dsn, sql_ios)
    #     print ios_res[0]

    #     # Android
    #     _pg_dsn = pg_settings('aa_android')
    #     sql_gp = "SELECT COUNT(1) FROM app where created <= '{}'".format(gp_max_date)
    #     gp_res = query(_pg_dsn, sql_gp)
    #     print gp_res[0]

    #     self.assertEqual(ios_max_created, ios_res[0][0], msg="IOS Failed: uni_count={}, db_count={}"
    #                      .format(ios_max_created, ios_res[0][0]))
    #     self.assertEqual(gp_max_created, gp_res[0][0], msg="Google Play Failed: uni_count={}, db_count={}"
    #                      .format(gp_max_created, gp_res[0][0]))
    
    # def test_sample_between_ios_raw_with_unified(self):
    #     _pg_dsn = pg_settings('aa')
    #     sql_ios = "SELECT id, company_id FROM aa_app LIMIT 1"
    #     ios_db = query(_pg_dsn, sql_ios)

    #     uni_df = self.spark.read.parquet(self.base_path)
    #     uni_df.createOrReplaceTempView('app_detail')
    #     ios_uni = self.spark.sql(
    #         "SELECT publisher_id "
    #         "FROM app_detail "
    #         "WHERE id='{}'".format(ios_db[0][0])
    #     ).collect()

    #     self.assertEqual(ios_uni[0][0], ios_db[0][1], msg="Failed -- db:{}, unified:{}".format(ios_db, ios_uni))

    def test_sample_from_ios_raw_with_unified(spark):
        sample_sql = "SELECT * FROM aa_app LIMIT 1"
        raw_data = query(pg_settings('aa'), sample_sql)
        unified_data = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/'
                                          'store.app.v1/dimension/market_code=apple-store/').filter("id = {}".format(raw_data[0][0])).collect()
        for i in range(16, len(raw_data[0]) - 1):
            unified_data_result = unified_data[0][IOS_FIELDS_MAPPING[i]]
            if isinstance(raw_data[0][i], (date, datetime)):
                assert str(raw_data[0][i]) == str(unified_data_result), "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result)
            elif i == 12 and raw_data[0][i]:
                unified_country_id = get_country_id_from_country_code(1, unified_data_result)
                assert raw_data[0][i] == unified_country_id, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_country_id)
            elif i == 15 and raw_data[0][i]:
                raw_category_id = get_ios_unified_category_id(raw_data[0][i])
                assert raw_category_id == unified_data_result, "raw: {}, unified: {}".format(
                    raw_category_id, unified_data_result)
            elif i == 16 and raw_data[0][i]:
                for item in unified_data_result:
                    assert item == unicode(raw_data[0][i]), "raw: {}, unified: {}".format(
                        raw_data[0][i], unified_data_result)
            elif isinstance(raw_data[0][i], str):
                assert raw_data[0][i] == "{}".format(unified_data_result.encode('utf-8')), \
                    "raw: {}, unified: {}".format(raw_data[0][i], unified_data_result)
            elif isinstance(raw_data[0][i], int):
                assert raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result)
            elif isinstance(raw_data[0][i], float):
                assert raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result)
            elif not raw_data[0][i]:
                assert unified_data_result is u"" or unified_data_result is None, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result)
            else:
                assert raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result)


unittest.main(argv=[''], verbosity=2)
%tb

In [0]:

ios_uni = spark.sql(
    "SELECT publisher_id "
    "FROM app_detail "
    "WHERE id='{}'".format('794883658')
).collect()

In [0]:

ios_uni[0][0]

In [0]:

from datetime import date, datetime
def test_sample_from_ios_raw_with_unified(spark):
    sample_sql = "SELECT * FROM aa_app LIMIT 1"
    raw_data = query(pg_settings('aa'), sample_sql)
    unified_data = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-store-free/unified/'
                                      'store.app.v1/dimension/market_code=apple-store/').filter("id = {}".format(raw_data[0][0])).collect()
    for i in range(16, len(raw_data[0]) - 1):
        unified_data_result = unified_data[0][IOS_FIELDS_MAPPING[i]]
        if isinstance(raw_data[0][i], (date, datetime)):
            assert str(raw_data[0][i]) == str(unified_data_result), "raw: {}, unified: {}".format(
                raw_data[0][i], unified_data_result)
        elif i == 12 and raw_data[0][i]:
            unified_country_id = get_country_id_from_country_code(1, unified_data_result)
            assert raw_data[0][i] == unified_country_id, "raw: {}, unified: {}".format(
                raw_data[0][i], unified_country_id)
        elif i == 15 and raw_data[0][i]:
            raw_category_id = get_ios_unified_category_id(raw_data[0][i])
            assert raw_category_id == unified_data_result, "raw: {}, unified: {}".format(
                raw_category_id, unified_data_result)
        elif i == 16 and raw_data[0][i]:
            for item in unified_data_result:
                assert item == unicode(raw_data[0][i]), "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result)
        elif isinstance(raw_data[0][i], str):
            assert raw_data[0][i] == "{}".format(unified_data_result.encode('utf-8')), \
                "raw: {}, unified: {}".format(raw_data[0][i], unified_data_result)
        elif isinstance(raw_data[0][i], int):
            assert raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                raw_data[0][i], unified_data_result)
        elif isinstance(raw_data[0][i], float):
            assert raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                raw_data[0][i], unified_data_result)
        elif not raw_data[0][i]:
            assert unified_data_result is u"" or unified_data_result is None, "raw: {}, unified: {}".format(
                raw_data[0][i], unified_data_result)
        else:
            assert raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                raw_data[0][i], unified_data_result)
test_sample_from_ios_raw_with_unified(spark)

In [0]:

IOS_FIELDS_MAPPING = {0: 'id',
                      1: 'name',
                      2: 'publisher',
                      3: 'release_date',
                      4: 'artwork_url',
                      5: 'slug',
                      6: 'scrape_reviews',
                      7: 'price',
                      8: 'version',
                      9: 'description',
                      10: 'date_scraped',
                      11: 'scrape_failed',
                      12: 'country_code',
                      13: 'dead',
                      14: 'publisher_id',
                      15: 'category_id',
                      16: 'ss_urls',
                      17: 'sku',
                      18: 'size',
                      19: 'req_version',
                      20: 'req_device',
                      21: 'web_urls',
                      22: 'has_game_center',
                      23: 'is_mac',
                      24: 'languages',
                      25: 'created',
                      26: 'support_url',
                      27: 'license_url',
                      28: 'content_rating',
                      29: 'link_apps',
                      30: 'scrape_review_delay',
                      31: 'requirements',
                      32: 'privacy_policy_url',
                      33: 'app_store_notes',
                      34: 'last_updated',
                      35: 'has_iap',
                      36: 'status',
                      37: 'bundle_id',
                      38: 'product_type',
                      39: 'bundle_product_count',
                      40: 'family_sharing',
                      41: 'purchased_separately_price',
                      42: 'seller',
                      43: 'required_devices',
                      44: 'current_release_date',
                      45: 'original_price',
                      46: 'sensitive_status',
                      47: 'has_imsg',
                      48: 'is_hidden_from_springboard',
                      49: 'subtitle',
                      50: 'promotional_text',
                      51: 'editorial_badge_type',
                      52: 'editorial_badge_name',
                      53: 'only_32_bit'}


GOOGLE_PLAY_FIELDS_MAPPING = {
    0: 'id',
    1: 'class',
    2: 'name',
    3: 'publisher',
    4: 'description',
    5: 'price',
    6: 'installs',
    7: 'version',
    8: 'release_date',
    9: 'require_os',
    10: 'category_id',
    11: 'content_rating',
    12: 'downloads_chart_url',
    13: 'ss_urls',
    14: 'video_url',
    15: 'icon_url',
    16: 'banner_image_url',
    17: 'country_code',
    18: 'permissions',
    19: 'whats_new',
    20: 'status',
    21: 'created',
    22: 'last_updated',
    23: 'web_urls',
    24: 'related_apps',
    25: 'also_installed_apps',
    26: 'more_from_developer_apps',
    27: 'is_publisher_top',
    28: 'privacy_policy_url',
    29: 'publisher_email',
    30: 'scrape_review_status',
    31: 'has_iap',
    32: 'publisher_code',
    33: 'publisher_id',
    34: 'current_release_date',
    35: 'size',
    36: 'original_price',
    37: 'sensitive_status',
    38: 'source'
}

In [0]:

import unittest
from datetime import date, datetime
from applications.db_check_v1.common.db_check_utils import query, pg_settings
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE, IOS_FIELDS_MAPPING,\
    GOOGLE_PLAY_FIELDS_MAPPING, CATEGORY_ID_MAPPING_BY_MARLKET_AND_DEVICE_CODE
from aadatapipelinecore.core.utils.spark import create_spark
def get_country_id_from_country_code(platform, country_code):
    country_id = None
    for key, _ in COUNTRY_CODE_MAPPING_BY_MARKET_CODE[platform].items():
        if unicode(COUNTRY_CODE_MAPPING_BY_MARKET_CODE[platform][key]) == country_code:
            country_id = key
    return country_id
def get_ios_unified_category_id(key):
    return CATEGORY_ID_MAPPING_BY_MARLKET_AND_DEVICE_CODE['apple-store']['ios-all']\
        .get(int(key), "Can't find ios unified category id mapping from category_id:{}.".format(key))
def get_gp_unified_category_id_from_id(key):
    return CATEGORY_ID_MAPPING_BY_MARLKET_AND_DEVICE_CODE['google-play']['android-all'].\
        get(int(key), "Can't find gp unified category id mapping from category_id:{}.".format(key))
class TestGameIQMarketSizeFullRestatement(PipelineTest):
    trigger_date_config = ('* * * * *', 0)  # set 2 hours buffer
    base_path = "s3://b2c-prod-data-pipeline-unified-store-free/unified/store.app.v1/dimension/"
    @classmethod
    def setUpClass(cls):
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext

    def test_sample_between_ios_raw_with_unified(self):
        sample_sql = "SELECT * FROM aa_app LIMIT 1"
        raw_data = query(pg_settings('aa'), sample_sql)
        unified_data = self.spark.read.parquet(self.base_path + 'market_code=apple-store/').filter(
            "id = {}".format(raw_data[0][0])).collect()
        for i in xrange(0, len(raw_data[0]) - 1):
            unified_data_result = unified_data[0][IOS_FIELDS_MAPPING[i]]
            if isinstance(raw_data[0][i], (date, datetime)):
                self.assertEqual(str(raw_data[0][i])[:10], str(unified_data_result)[:10], "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))
            elif i == 12 and raw_data[0][i]:
                unified_country_id = get_country_id_from_country_code('apple-store', unified_data_result)
                self.assertEqual(raw_data[0][i], unified_country_id, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_country_id))
            elif i == 15 and raw_data[0][i]:
                raw_category_id = get_ios_unified_category_id(raw_data[0][i])
                self.assertEqual(raw_category_id, unified_data_result, "raw: {}, unified: {}".format(
                    raw_category_id, unified_data_result))
            elif i == 16 and raw_data[0][i]:
                for item in unified_data_result:
                    self.assertEqual(item, unicode(raw_data[0][i]), "raw: {}, unified: {}".format(
                        raw_data[0][i], unified_data_result))
            elif isinstance(raw_data[0][i], str):
                format_uni = "{}".format(unified_data_result.encode('utf-8'))
                self.assertEqual(raw_data[0][i], format_uni,
                                 "raw: {}, unified: {}".format(raw_data[0][i], format_uni))
            elif isinstance(raw_data[0][i], int):
                self.assertEqual(raw_data[0][i], unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))
            elif isinstance(raw_data[0][i], float):
                self.assertEqual(raw_data[0][i], unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))
            elif not raw_data[0][i]:
                self.assertTrue(unified_data_result is u"" or unified_data_result is None, "raw: {}, unified: {}"
                                .format(raw_data[0][i], unified_data_result))
            else:
                self.assertTrue(raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))

    # pylint: disable=R0912, R1260

    def test_sample_between_gp_raw_with_unified(self):
        sample_sql = "SELECT * FROM app LIMIT 1"
        raw_data = query(pg_settings('aa_android'), sample_sql)
        unified_data = self.spark.read.parquet(self.base_path + 'market_code=google-play/').filter(
            "id = {}".format(raw_data[0][0])).collect()
        for i in xrange(0, len(raw_data[0]) - 1):
            unified_data_result = unified_data[0][GOOGLE_PLAY_FIELDS_MAPPING[i]]
            if isinstance(raw_data[0][i], (date, datetime)):
                self.assertTrue(str(raw_data[0][i])[:10] == str(unified_data_result)[:10], "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))
            elif isinstance(raw_data[0][i], list) and raw_data[0][i]:
                for item in raw_data[0][i]:
                    self.assertTrue(unicode(item) in unified_data_result, "raw: {}, unified: {}".format(
                        raw_data[0][i], unified_data_result))
            elif i == 10 and raw_data[0][10]:
                raw_category_id = get_gp_unified_category_id_from_id(raw_data[0][10])
                self.assertTrue(raw_category_id == unified_data_result, "raw: {}, unified: {}".format(
                    raw_category_id, unified_data_result))
            elif i == 13 and raw_data[0][i]:
                for item in unified_data_result:
                    self.assertTrue(item == unicode(raw_data[0][i]), "raw: {}, unified: {}".format(
                        raw_data[0][i], unified_data_result))
            elif i == 17 and raw_data[0][i]:  # store_id mapping to country_code
                self.assertTrue(raw_data[0][i] == get_country_id_from_country_code('google-play', unified_data_result),
                                "raw: {}, unified: {}".format(raw_data[0][i], unified_data_result))
            elif isinstance(raw_data[0][i], str):
                format_uni = "{}".format(unified_data_result.encode('utf-8'))
                self.assertTrue(raw_data[0][i] == "{}".format(unified_data_result.encode('utf-8')),
                                "raw: {}, unified: {}".format(raw_data[0][i], format_uni))
            elif isinstance(raw_data[0][i], int):
                self.assertTrue(raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))
            elif isinstance(raw_data[0][i], float):
                self.assertTrue(raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))
            elif not raw_data[0][i]:
                self.assertTrue(not unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))
            else:
                self.assertTrue(raw_data[0][i] == unified_data_result, "raw: {}, unified: {}".format(
                    raw_data[0][i], unified_data_result))

unittest.main(argv=[''], verbosity=2)
%tb

In [0]:

a = [ 'id'
, 'name'
, 'company'
, 'release_date'         
, 'artwork_url'                 
, 'slug'
, 'scrape_reviews'
, 'price' 
, 'version'                   
, 'description'                 
, 'date_scraped'             
, 'scrape_failed'              
, 'store_id'                  
, 'dead'               
, 'company_id'                  
, 'category_id'                
, 'ss_urls'                     
, 'sku'                       
, 'size'                      
, 'req_version'                
, 'req_device'                     
, 'web_urls'                 
, 'has_game_center'
, 'is_mac'                     
, 'languages'                   
, 'created'                  
, 'support_url'                 
, 'license_url'                 
, 'content_rating'              
, 'link_apps'                   
, 'scrape_review_delay'         
, 'requirements'                
, 'privacy_policy_url'         
, 'app_store_notes'             
, 'last_updated'              
, 'has_iap'                    
, 'status '                      
, 'bundle_id'                
, 'product_type    '               
, 'bundle_product_count   '       
, 'family_sharing      '        
, 'purchased_separately_price on   '       
, 'seller                '   
, 'required_devices     '        
, 'current_release_date '       
, 'original_price      '
, 'sensitive_status     '         
, 'has_imsg                   ' 
, 'is_hidden_from_springboard  '
, 'subtitle                '    
, 'promotional_text       '     
, 'editorial_badge_type     ' 
, 'editorial_badge_name    '    
, 'only_32_bit'
]
len(a)

In [0]:

len(IOS_FIELDS_MAPPING)

In [0]:

sample_sql = "SELECT * FROM aa_app LIMIT 1"
raw_data = query(pg_settings('aa'), sample_sql)
len(raw_data[0])

In [0]:

b = ['id'
, 'class'
, 'name'                 
, 'company'                
, 'description'               
, 'price'              
, 'installs'                 
, 'version'                   
, 'release_date   '          
, 'require_os     '           
, 'category_id      '         
, 'content_rating    '        
, 'downloads_chart_url  '     
, 'ss_urls         '         
, 'video_url     '            
, 'icon_url   '               
, 'banner_image_url     '     
, 'store_id      '            
, 'permissions  '             
, 'whats_new    '             
, 'status'
, 'created'
, 'last_updated'
, 'web_urls     '             
, 'related_apps        '     
, 'also_installed_apps    '  
, 'more_from_developer_apps '
, 'is_publisher_top       '   
, 'privacy_policy_url    '    
, 'publisher_email   '        
, 'scrape_review_status'
, 'has_iap     '              
, 'company_code   '           
, 'company_id '               
, 'current_release_date  '    
, 'size         '             
, 'original_price    '        
, 'sensitive_status'
, 'source     '               
, 'contains_ads ']             

In [0]:

b = [i.strip() for i in b]
print b

In [0]:

bb = []
for k, v in GOOGLE_PLAY_FIELDS_MAPPING.items():
    bb.append(v)
print bb

In [0]:

print len(b), len(bb)

In [0]:

for i in bb:
    if i not in b:
        print i

In [0]:

for i in b:
    if i not in bb:
        print i